# TP1: Réseau de neurones à deux couches, fonction de perte **Entropie croisée**

In [22]:
import random
import numpy as np
from utils.data_utils import load_CIFAR10
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# pour automatiquement recharger les modules externes
# voir http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<font size=5>MLP à deux couches</font>

Ici, nous développerons un **réseau pleinement connecté** à deux couches avec **softmax** et **entropie croisée**.  En principe, vous devriez avoir pris connaissance du notebook "tp1_simple_neural_net.ipynb".  Le but ici est d'enchasser les différents éléments d'un réseau de neurones dans des **classes**.

Au début, nous commencerons avec un petit réseau à 

* **10 neurones cachées**
* **3 classes** 
* un vecteur d'entrée de taille **4**.

Avant de commencer, vous devez vous familier avec les classes **model.Model**, **layers.Dense** et la loss **cross_entropy_loss** (cette dernière fonction comprend le softmax + la loss)

Le code à produire se situe : 

* dans la classe **Dense** (répertoire layer)
* dans la fonction **cross_entropy_loss** (dans utils.model_loss)
* dans le fichier **activations.py** (répetoire utils)

À ce point-ci, l'entropie croisée (et son gradient) que vous avez codé précédemment **doit être fonctionnelle**.  Ce code peut donc en bonne partie être récupéré ici!

In [23]:
##############################################################################
# On crée un modèle jouet pour rapidement tester les différentes composantes #
# La classe Model encapsule et relie les différentes couches de notre        #
# réseau de neurones.                                                        #
##############################################################################
from model.Model import Model
from layers.Dense import Dense
from utils.model_loss import cross_entropy_loss

hidden_size = 10
num_classes = 3
input_size = 4

def create_toy_model():
    np.random.seed(0)
    model = Model()
    model.add(Dense(input_size, hidden_size, weight_scale=1e-1, activation='relu'))
    model.add(Dense(hidden_size, num_classes, weight_scale=1e-1))
    model.add_loss(cross_entropy_loss)
    return model

model = create_toy_model()

In [24]:
def create_random_toy_data(nb_elements, input_size, nb_classes):
    np.random.seed(1)
    
    X = 10 * np.random.randn(nb_elements, input_size)
    y = np.random.randint(nb_classes, size=nb_elements)
        
    return X, y

In [25]:
x,y=create_random_toy_data(1, 5, 3)
x.shape

(1, 5)

### Commençons avec la prédiction d'**un seul vecteur d'entrée** de taille 4 et dont la cible est 0.  Ici, le **score** est la sortie du réseau avant la **softmax**

In [26]:
##############################################################################
# TODO: Implémenter la méthode forward (propagation avant) de la classe de   #
# couche Dense.                                                              #
##############################################################################
X, y = create_random_toy_data(1, input_size, num_classes)

scores = model.forward(X)
print('Your scores:')
print(scores)
print()
print('correct scores:')
correct_scores = np.asarray([[-0.81233741, -1.27654624, -0.70335995]])
print(correct_scores)
print()

# La différence devrait être assez basse, en principe inférieure à 1e-7.
print('Difference between your scores and correct scores:')
print(np.sum(np.abs(scores - correct_scores)))

Your scores:
[[-0.81233741 -1.27654624 -0.70335995]]

correct scores:
[[-0.81233741 -1.27654624 -0.70335995]]

Difference between your scores and correct scores:
5.897841770519108e-09


In [27]:
##############################################################################
# TODO: Implémenter le calcul de perte vectorisé adapté pour un réseau de    #
# neurones dans la fonction softmax_loss. La différence avec la softmax      #
# implémentée dans la partie précédente est que la dérivée retournée par     #
# softmax_loss est en fonction des scores, et non en fonction des poids      #
##############################################################################

loss, _, softmax = model.calculate_loss(scores, y, 0.1)
print('Softmax: ', softmax)
print('Loss: ', loss)

correct_loss = 0.938286073616293
correct_softmax = np.asarray([0.3644621, 0.22911264, 0.40642526])

print('\nCorrect loss: ', correct_loss)
print('Difference between your loss and correct loss:')
# on devrait obtenir une erreur de loss inférieure à 1e-9.
print(np.sum(np.abs(loss - correct_loss)))

# on devrait obtenir une erreur de softmax inférieure à 1e-7.
print('\nDifference between your softmax and correct softmax:')
print(np.sum(np.abs(correct_softmax - softmax)))

Softmax:  [[0.3644621  0.22911264 0.40642526]]
Loss:  0.9382860736162932

Correct loss:  0.938286073616293
Difference between your loss and correct loss:
2.220446049250313e-16

Difference between your softmax and correct softmax:
8.204646462228737e-09


### Maintenant allons-y avec la prédiction de **5 vecteurs d'entrée** de taille 4 et dont la cible est 0.  Ici, le **score** est la sortie du réseau avant la **softmax**

In [28]:
##############################################################################
# TODO: Implémenter la méthode forward (propagation avant) de la classe de   #
# couche Dense.                                                              #
##############################################################################
X, y = create_random_toy_data(5, input_size, num_classes)

scores = model.forward(X)
print('Your scores:')
print(scores)
print()
print('Correct scores:')
correct_scores = np.asarray([
  [-0.81233741, -1.27654624, -0.70335995],
  [-0.17129677, -1.18803311, -0.47310444],
  [-0.51590475, -1.01354314, -0.8504215 ],
  [-0.15419291, -0.48629638, -0.52901952],
  [-0.00618733, -0.12435261, -0.15226949]])
print(correct_scores)
print()

# La différence devrait être assez basse, en principe inférieure à 1e-7.
print('Difference between your scores and correct scores:')
print(np.sum(np.abs(scores - correct_scores)))

Your scores:
[[-0.81233741 -1.27654624 -0.70335995]
 [-0.17129677 -1.18803311 -0.47310444]
 [-0.51590475 -1.01354314 -0.8504215 ]
 [-0.15419291 -0.48629638 -0.52901952]
 [-0.00618733 -0.12435261 -0.15226949]]

Correct scores:
[[-0.81233741 -1.27654624 -0.70335995]
 [-0.17129677 -1.18803311 -0.47310444]
 [-0.51590475 -1.01354314 -0.8504215 ]
 [-0.15419291 -0.48629638 -0.52901952]
 [-0.00618733 -0.12435261 -0.15226949]]

Difference between your scores and correct scores:
3.6802720745909845e-08


In [29]:
##############################################################################
# TODO: Implémenter le calcul de perte vectorisé adapté pour un réseau de    #
# neurones dans la fonction softmax_loss. La différence avec la softmax      #
# implémentée dans la partie précédente est que la dérivée retournée par     #
# softmax_loss est en fonction des scores, et non en fonction des poids      #
#                                                                            #
# Code à modifier dans "calculate_loss" du fichier model_loss.py             #
##############################################################################

loss, _, softmax = model.calculate_loss(scores, y, 0.1)
print('Softmax: ', softmax)
print('Loss: ', loss)

correct_loss = 1.06368300738

correct_softmax = np.asarray([
    [0.3644621,  0.22911264, 0.40642526],
    [0.47590629, 0.17217039, 0.35192332],
    [0.43035767, 0.26164229, 0.30800004],
    [0.41583127, 0.2983228,  0.28584593],
    [0.36328815, 0.32279939, 0.31391246]])

# on devrait obtenir une erreur de loss inférieure à 1e-10.
print('\nCorrect loss: ', correct_loss)
print('Difference between your loss and correct loss:')
print(np.sum(np.abs(loss - correct_loss)))

# on devrait obtenir une erreur de softmax inférieure à 1e-7.
print('\nDifference between your softmax and correct softmax:')
print(np.sum(np.abs(softmax - correct_softmax)))


Softmax:  [[0.3644621  0.22911264 0.40642526]
 [0.47590629 0.17217039 0.35192332]
 [0.43035767 0.26164229 0.30800004]
 [0.41583127 0.2983228  0.28584593]
 [0.36328815 0.32279939 0.31391246]]
Loss:  1.0636830073861907

Correct loss:  1.06368300738
Difference between your loss and correct loss:
6.190603585309873e-12

Difference between your softmax and correct softmax:
2.9173411658645065e-08


# Rétropropagation

Maintenant vous devez rédiger la **rétro-progatation** de la classe **model**.  Vos gradients seront testés avec un gradient numérique.

In [111]:
##############################################################################
# TODO: Implémenter la méthode backward (rétro-propagation) de la classe de  #
# couche Dense, ainsi que la méthode backward de la classe Model.            #
##############################################################################
loss, dScores, softmax = model.calculate_loss(scores, y, 0.1)

_ = model.backward(dScores)  # les gradients sont stockés dans les objets "layers" du modèle

In [112]:
from utils.gradients import evaluate_numerical_gradient

# Retourne l'erreur relative maximale des matrices de gradients passées en paramètre.
# Pour chaque paramètre, l'erreur relative devrait être inférieure à environ 1e-8.
def rel_error(x, y):
    rel = np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y)))
    return np.max(rel)

gradients = model.gradients()
model_params = model.parameters()

# Si tout va bien, vous devriez avoir des erreurs inférieurs à 1e-8
for layer_name, layer_params in model_params.items():
    for param_name, _ in layer_params.items():
        grad_num = evaluate_numerical_gradient(X, y, model, layer_name, param_name, reg=0.1)
        max_error = rel_error(grad_num, gradients[layer_name][param_name])
        print('%s max relative error: %e' % (layer_name + '-' + param_name, max_error))

L0-W max relative error: 1.000000e+00
L0-b max relative error: 8.635600e-01
L1-W max relative error: 9.408715e-01
L1-b max relative error: 9.849827e-01


In [101]:
print(evaluate_numerical_gradient(X, y, model, 'L1', 'W', reg=0.1))
print()
print(gradients['L1']['W'])

[[-5.10751521e-01  1.60804380e-01  3.08198730e-01]
 [ 1.95077540e-02 -5.09652182e-03 -4.38074302e-03]
 [-5.28612456e-02 -1.37742710e-01  1.69711927e-01]
 [-5.44467550e-01 -3.12697338e-01  8.49951845e-01]
 [-3.12752322e-01 -5.22542752e-01  8.18098879e-01]
 [ 4.28331871e-03  6.65172217e-04  3.02471898e-03]
 [-6.34322094e-03 -3.62741166e-03 -6.72460448e-03]
 [-3.59553162e-03 -8.13146283e-03 -1.72628260e-02]
 [-6.89155934e-02 -6.91175498e-01  7.41545560e-01]
 [-3.71809831e-01 -4.73581814e-01  8.41465938e-01]]

[[-2.52436494e+00  8.92058200e-01  1.59055833e+00]
 [ 1.95077540e-02 -5.09652182e-03 -4.38074302e-03]
 [-4.12337144e-01 -5.73737528e-01  9.65182644e-01]
 [-3.65481669e+00 -8.85695784e-01  4.53329943e+00]
 [-3.20620555e+00 -1.51091813e+00  4.69992749e+00]
 [ 4.28331871e-03  6.65172224e-04  3.02471898e-03]
 [-6.34322094e-03 -3.62741166e-03 -6.72460448e-03]
 [-3.59553162e-03 -8.13146282e-03 -1.72628260e-02]
 [-2.26212757e+00 -2.19630676e+00  4.43988880e+00]
 [-3.98300971e+00 -1.00947176

# Entraînement

Il est maintanant temps de rédiger le code de la descente de gradient devant permettre d'entraîner le modèle.  Le code à rédiger est dans le fichier **model/Solver.py**.  Si votre code fonctionne, le graphique de la loss devrait **descendre**.

In [80]:
from visualization.utils import visualize_loss
from model.Solver import solver, SGD

model = create_toy_model()
sgd_optimizer = SGD(1e-1, model)

#
# TODO
# Ajouter code de descente de gradient dans la fonction solver du fichier Solver.py
#
loss_history, _, _ = solver(X, y, X, y, 1e-2, sgd_optimizer, num_iter=100,batch_size=2)

print('Final training loss: ', loss_history[-1])

# Visualisation de l'historique de perte lors de l'entraînement
visualize_loss(loss_history)

ValueError: non-broadcastable output operand with shape (10,) doesn't match the broadcast shape (1,10)

<font size="5">Classifieur d'images</font>

Si votre code fonctionne bien, vous devriez être capable d'entraîner un modèle sur CIFAR10

In [ ]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Charger la banque de données CIFAR-10, prétraiter les images et ajouter une dimension pour le biais.
    
    Input :
    - num_training : nombre d'images à mettre dans l'ensemble d'entrainement
    - num_validation : nombre d'images à mettre dans l'ensemble de validation
    - num_test : nombre d'images à mettre dans l'ensemble de test
    - num_dev : d'images à mettre dans l'ensemble dev
    
    Output :
    - X_train, y_train : données et cibles d'entrainement
    - X_val, y_val: données et cibles de validation
    - X_test y_test: données et cibles de test 
    - X_dev, y_dev: données et cicles dev
    """
    # Charger les données CIFAR-10
    cifar10_dir = 'datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
  
    # Séparer en ensembles d'entraînement, de validation, de test et de dev
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]

    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))

    # Normalisation
    X_train -= np.mean(X_train, axis = 0)
    X_val -= np.mean(X_val, axis = 0)
    X_test -= np.mean(X_test, axis = 0)
    X_dev -= np.mean(X_dev, axis = 0)

    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev

In [ ]:
# Charger les images pour les différents ensembles de données
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

In [ ]:
# Si le modèle et toutes ses composantes ont bien été implémentées
# à l'étape du modèle jouet,
def create_model(hidden_size):
    model = Model()
    model.add(Dense(dim_input=3*32*32, dim_output=hidden_size, activation='relu'))
    model.add(Dense(dim_input=hidden_size, dim_output=10))
    model.add_loss(cross_entropy_loss)
    return model
    
model = create_model(50)

scores = model.forward(X_dev)

In [ ]:
loss, dScores, softmax = model.calculate_loss(scores, y_dev, 0.5)

# La loss d'un modèle non-entrainé devrait s'approcher de -log(0.1).
print('loss: %f' % loss)
print('sanity check loss: %f' % (-np.log(0.1)))

print('dScores shape: ', dScores.shape)
print('softmax shape: ', softmax.shape)

In [ ]:
optimizer = SGD(8e-5, model)

loss_history, train_accuracy, val_accuracy = solver(X_train, y_train, X_val, y_val, 0.5, optimizer, lr_decay=0.98, num_iter=3000)

print('Final training loss: ', loss_history[-1])

<font size=5>Visualisation</font>

In [ ]:
from visualization.utils import visualize_loss

# Visualisation de l'historique de loss de l'entraînement
visualize_loss(loss_history)

In [ ]:
from visualization.utils import visualize_accuracy

# Visualisation de l'historique d'accuracy lors de l'entraînement.
# Ceci inclut la accuracy d'entraînement et la accuracy de validation
visualize_accuracy(train_accuracy, val_accuracy)

# Si la accuracy de validation est proche de 0.4, c'est bon signe

In [ ]:
from visualization.utils import visualize_as_grid

def show_net_weights(model):
    W1 = model.parameters()['L0']['W']
    W1 = W1.reshape(32, 32, 3, -1).transpose(3, 0, 1, 2)
    plt.imshow(visualize_as_grid(W1, padding=3).astype('uint8'))
    plt.gca().axis('off')
    plt.show()

show_net_weights(model)

## Recherche d'hyper-paramètres

Tout comme dans le notebook précédent, veuillez rédiger du code afin de trouver les meilleurs taux d'apprentissage et terme de régularisation.

In [ ]:
best_model = None 
best_val_acc = 0.0
best_loss = None
best_train_acc = 0.0
best_params = None
best_model = None

from itertools import product as itprod

lr_list = 10 ** np.linspace(-4, -3, 4)
reg_list = 10 ** np.linspace(-7, -3, 4)
hs = 200
nb_iter = 1000

# Mettre les résultats des meilleurs hyper-paramètres dans les variables *best_ABC* définis ci-haut.
# Avec 1000 iterations par entrainement, vous devriez obtenir une justesse de validation de 39%.
# TODO
# Mettre code ici.

In [ ]:
print('Best: {}'.format(best_params))
# Visualisation de l'historique de perte lors de l'entraînement
visualize_loss(best_loss)

# Visualisation de l'historique d'accuracy lors de l'entraînement.
# Ceci inclut la accuracy d'entraînement et la accuracy de validation
visualize_accuracy(best_train_acc, best_val_acc)

In [ ]:
show_net_weights(best_model)

In [ ]:
# Si tout va bien, vous devriez obtenir un justesse sur l'ensemble de test de 45% ou plus.
test_acc = (best_model.predict(X_test) == y_test).mean()
print('Test accuracy: ', test_acc)